In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/Reddit.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,class,text
0,0,1,subsection retarded hungarians ohh boy brace l...
1,1,0,hiii got work 444 mainly typa guys imagine wr...
2,2,0,wow guess soyboys every country
3,3,0,owen benjamin soyboy song goes every country ...
4,4,0,y all hear sumn means live small town for wor...


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

34259


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001,0.001,0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
16072/16072 [==============================] - 120s 7ms/step - loss: 0.4301 - acc: 0.8212
Epoch 2/3
16072/16072 [==============================] - 119s 7ms/step - loss: 0.3157 - acc: 0.8786
Epoch 3/3
16072/16072 [==============================] - 119s 7ms/step - loss: 0.2814 - acc: 0.8978
Epoch 1/3
16072/16072 [==============================] - 119s 7ms/step - loss: 0.4272 - acc: 0.8197
Epoch 2/3
16072/16072 [==============================] - 118s 7ms/step - loss: 0.3192 - acc: 0.8766
Epoch 3/3
16072/16072 [==============================] - 119s 7ms/step - loss: 0.2875 - acc: 0.8943
Epoch 1/3
16072/16072 [==============================] - 118s 7ms/step - loss: 0.4246 - acc: 0.8263
Epoch 2/3
16072/16072 [==============================] - 122s 8ms/step - loss: 0.3167 - acc: 0.8807
Epoch 3/3
16072/16072 [==============================] - 155s 10ms/step - loss: 0.2848 - acc: 0.8985
Epoch 1/3
16072/16072 [==============================] - 214s 13ms/step - loss: 0.3783 - acc: 0.852

In [8]:
print(score)

[[0.8506032609887052, 0.84999444999445, 0.8485997310932095], [0.8649756433875471, 0.8715189873417721, 0.8678592969399019], [0.8590881636453679, 0.8612604881636816, 0.8614428548015813]]


In [10]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(100, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Reddit.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Reddit.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Reddit.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Reddit.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
16072/16072 [==============================] - 127s 8ms/step - loss: 0.3409 - acc: 0.8763
Epoch 2/5
16072/16072 [==============================] - 117s 7ms/step - loss: 0.2690 - acc: 0.9111
Epoch 3/5
16072/16072 [==============================] - 116s 7ms/step - loss: 0.2514 - acc: 0.9139
Epoch 4/5
16072/16072 [==============================] - 115s 7ms/step - loss: 0.2272 - acc: 0.9185
Epoch 5/5
16072/16072 [==============================] - 108s 7ms/step - loss: 0.1892 - acc: 0.9279
Saved model to disk
Fold Number: 1
Epoch 1/5
16073/16073 [==============================] - 122s 8ms/step - loss: 0.3304 - acc: 0.8828
Epoch 2/5
16073/16073 [==============================] - 128s 8ms/step - loss: 0.2651 - acc: 0.9110
Epoch 3/5
16073/16073 [==============================] - 142s 9ms/step - loss: 0.2539 - acc: 0.9136
Epoch 4/5
16073/16073 [==============================] - 120s 7ms/step - loss: 0.2273 - acc: 0.9203
Epoch 5/5
16073/16073 [===========================

In [11]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Accuracies: 
[0.9049514804677781, 0.899701343952215, 0.9143852663016426, 0.8989547038327527, 0.9066699850671976]
Mean: 0.9049325559243172
Std Dev: 0.005576254046726334
Macro-F1-Scores: 
[0.8674855803656818, 0.8569249521648524, 0.8793137169983711, 0.8586266687743002, 0.8646372723595105]
Mean: 0.8653976381325432
Std Dev: 0.007952701973983893


In [5]:
# load json and create model
json_file = open('Models/model_Reddit.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_Reddit.h5")
print("Loaded model from disk")

Loaded model from disk


In [13]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f481ffd2e10> False
<keras.layers.recurrent.LSTM object at 0x7f481ffd28d0> True
<keras.layers.core.Dense object at 0x7f481ffd2a90> True
<keras.layers.core.Dense object at 0x7f481ffd2f98> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f481ffd2e10> False
<keras.layers.recurrent.LSTM object at 0x7f481ffd28d0> False
<keras.layers.core.Dense object at 0x7f481ffd2a90> True
<keras.layers.core.Dense object at 0x7f481ffd2f98> True
Epoch 1/8
4460/4460 [======

In [14]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9215598386373823, 0.9173016584491259, 0.9123318385650224, 0.912780269058296, 0.9168161434977579]
Mean: 0.9161579496415169
Std Dev: 0.0033754865072931034
Macro-F1-Scores: 
[0.8697590028661859, 0.8576095748527452, 0.8403288902928379, 0.8391982583503246, 0.8534286712213212]
Mean: 0.852064879516683
Std Dev: 0.01139270783081534
Without Fine Tuning: 
Accuracies: 
[0.5434782608695652, 0.5582698341550874, 0.5522421524663678, 0.5473094170403587, 0.5448430493273543]
Mean: 0.5492285427717467
Std Dev: 0.005418636942040396
Macro-F1-Scores: 
[0.5219445384608709, 0.5340646474161403, 0.5283907186171722, 0.5252687422906663, 0.5224397107695125]
Mean: 0.5264216715108725
Std Dev: 0.00446143933445894


In [15]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f481eb77780> False
<keras.layers.recurrent.LSTM object at 0x7f481f0b0438> True
<keras.layers.core.Dense object at 0x7f481eb779b0> True
<keras.layers.core.Dense object at 0x7f481ff680b8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f481eb77780> False
<keras.layers.recurrent.LSTM object at 0x7f481f0b0438> False
<keras.layers.core.Dense object at 0x7f481eb779b0> True
<keras.layers.core.Dense object at 0x7f481ff680b8> True
Epoch 1/8
2841/2841 [======

In [16]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.7470981357720717, 0.7512315270935961, 0.7526389866291344, 0.7533427163969036, 0.7652235128475888]
Mean: 0.7539069757478589
Std Dev: 0.006058193200408551
Macro-F1-Scores: 
[0.6393413380623258, 0.6661048277434433, 0.6922424510731324, 0.671123621862538, 0.7136177438038824]
Mean: 0.6764859965090644
Std Dev: 0.025073020940886436
Without Fine Tuning: 
Accuracies: 
[0.6918747801618009, 0.6914144968332161, 0.6921182266009852, 0.6914144968332161, 0.6902499120028159]
Mean: 0.691414382486407
Std Dev: 0.0006423945968701341
Macro-F1-Scores: 
[0.432012078308307, 0.4338047398102032, 0.4301196135790459, 0.42982367645245545, 0.4253006730663136]
Mean: 0.43021215624326503
Std Dev: 0.002843537367987427


In [17]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f481abe75c0> False
<keras.layers.recurrent.LSTM object at 0x7f481ac3b2b0> True
<keras.layers.core.Dense object at 0x7f481abe7a90> True
<keras.layers.core.Dense object at 0x7f482006c080> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f481abe75c0> False
<keras.layers.recurrent.LSTM object at 0x7f481ac3b2b0> False
<keras.layers.core.Dense object at 0x7f481abe7a90> True
<keras.layers.core.Dense object at 0x7f482006c080> True
Epoch 1/8
2383/2383 [======

In [18]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.739513422818792, 0.7343684431389006, 0.7423415862358371, 0.7259756609315988, 0.7373059169114562]
Mean: 0.735901006007317
Std Dev: 0.0056100913961562564
Macro-F1-Scores: 
[0.6610540616463901, 0.6580133866477677, 0.6701219985101896, 0.6424063875576371, 0.6676420681898794]
Mean: 0.6598475805103728
Std Dev: 0.009750207986424494
Without Fine Tuning: 
Accuracies: 
[0.6694630872483222, 0.6731011330255979, 0.6764582459085187, 0.6768778850188838, 0.6739404112463282]
Mean: 0.6739681524895301
Std Dev: 0.002672647144036071
Macro-F1-Scores: 
[0.416324587796515, 0.4212457969668013, 0.4349094599893889, 0.4361825690441366, 0.4316006523915344]
Mean: 0.42805261323767524
Std Dev: 0.007868677641417963


In [19]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f4819cc1320> False
<keras.layers.recurrent.LSTM object at 0x7f4819cc15f8> True
<keras.layers.core.Dense object at 0x7f4818df8780> True
<keras.layers.core.Dense object at 0x7f4818df8588> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f4819cc1320> False
<keras.layers.recurrent.LSTM object at 0x7f4819cc15f8> False
<keras.layers.core.Dense object at 0x7f4818df8780> True
<keras.layers.core.Dense object at 0x7f4818df8588> True
Epoch 1/8
1618/1618 [======

In [20]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8319950586781965, 0.8312731767614339, 0.8226205191594561, 0.8238566131025958, 0.8288009888751545]
Mean: 0.8277092713153674
Std Dev: 0.00382098459528334
Macro-F1-Scores: 
[0.517622728149044, 0.5116176475466115, 0.5344211671404007, 0.5152342776766821, 0.5681695772612309]
Mean: 0.5294130795547939
Std Dev: 0.020899163930941447
Without Fine Tuning: 
Accuracies: 
[0.762816553428042, 0.7404202719406675, 0.7564894932014833, 0.754017305315204, 0.7453646477132262]
Mean: 0.7518216543197246
Std Dev: 0.007989766754942524
Macro-F1-Scores: 
[0.5271238111464717, 0.5142905336828894, 0.5399906481631145, 0.5382737931350955, 0.5189749924954395]
Mean: 0.5277307557246022
Std Dev: 0.010189523447341142


In [21]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f47fe926b70> False
<keras.layers.recurrent.LSTM object at 0x7f47fe926d30> True
<keras.layers.core.Dense object at 0x7f47fe926dd8> True
<keras.layers.core.Dense object at 0x7f47fe0d71d0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f47fe926b70> False
<keras.layers.recurrent.LSTM object at 0x7f47fe926d30> False
<keras.layers.core.Dense object at 0x7f47fe926dd8> True
<keras.layers.core.Dense object at 0x7f47fe0d71d0> True
Epoch 1/8
4460/4460 [======

In [22]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9444195428059167, 0.9432989690721649, 0.942152466367713, 0.942152466367713, 0.9381165919282511]
Mean: 0.9420280073083518
Std Dev: 0.002129081315111752
Macro-F1-Scores: 
[0.5881655888622911, 0.5785604875304399, 0.6121221437037098, 0.5108185503618654, 0.5756358085383056]
Mean: 0.5730605157993224
Std Dev: 0.033659822013880514
Without Fine Tuning: 
Accuracies: 
[0.6125056028686687, 0.6290900941281936, 0.6298206278026905, 0.6181614349775785, 0.6311659192825112]
Mean: 0.6241487358119284
Std Dev: 0.007446322053736886
Macro-F1-Scores: 
[0.4481632941515437, 0.45957475178461354, 0.4608127313763736, 0.4575221063859131, 0.4583957953200146]
Mean: 0.45689373580369175
Std Dev: 0.004503732159207438


In [23]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f47e35d09b0> False
<keras.layers.recurrent.LSTM object at 0x7f47e35d02e8> True
<keras.layers.core.Dense object at 0x7f47e35dedd8> True
<keras.layers.core.Dense object at 0x7f4818d3ec18> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f47e35d09b0> False
<keras.layers.recurrent.LSTM object at 0x7f47e35d02e8> False
<keras.layers.core.Dense object at 0x7f47e35dedd8> True
<keras.layers.core.Dense object at 0x7f4818d3ec18> True
Epoch 1/8
1926/1926 [======

In [24]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8879668049792531, 0.8811007268951194, 0.887331256490135, 0.8904465212876428, 0.8883696780893042]
Mean: 0.8870429975482909
Std Dev: 0.00314932211768921
Macro-F1-Scores: 
[0.569882877858338, 0.5822699569535013, 0.588880527643752, 0.5330243803224587, 0.5241717619399461]
Mean: 0.5596459009435992
Std Dev: 0.02622375603616568
Without Fine Tuning: 
Accuracies: 
[0.8838174273858921, 0.8847352024922118, 0.8868120456905504, 0.8888888888888888, 0.8878504672897196]
Mean: 0.8864208063494526
Std Dev: 0.0018924731141589425
Macro-F1-Scores: 
[0.49448450229422225, 0.49496109197601734, 0.49599308573212014, 0.5170194615269236, 0.49233900687756565]
Mean: 0.49895942968136975
Std Dev: 0.009108299778938797


In [25]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f47e35d8cf8> False
<keras.layers.recurrent.LSTM object at 0x7f47e35d85f8> True
<keras.layers.core.Dense object at 0x7f47e303d780> True
<keras.layers.core.Dense object at 0x7f47cf0ed710> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f47e35d8cf8> False
<keras.layers.recurrent.LSTM object at 0x7f47e35d85f8> False
<keras.layers.core.Dense object at 0x7f47e303d780> True
<keras.layers.core.Dense object at 0x7f47cf0ed710> True
Epoch 1/8
6079/6079 [======

In [26]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9039631639532971, 0.9129934210526316, 0.9049185721335746, 0.8993255469649614, 0.9052475736140813]
Mean: 0.9052896555437092
Std Dev: 0.0044004010699268785
Macro-F1-Scores: 
[0.9011773112957104, 0.9103449189010262, 0.9024396867107516, 0.8965541748498006, 0.9026176263146537]
Mean: 0.9026267436143884
Std Dev: 0.004440320903103619
Without Fine Tuning: 
Accuracies: 
[0.900180891300773, 0.9072368421052631, 0.8980095410429347, 0.8960355321598947, 0.898667544003948]
Mean: 0.9000260701225626
Std Dev: 0.003843320911897874
Macro-F1-Scores: 
[0.8962752121533986, 0.9036949188757082, 0.8939667700390646, 0.8920574847090745, 0.8948451824943782]
Mean: 0.8961679136543248
Std Dev: 0.004003938002044542


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Reddit.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Reddit_model = model_from_json(loaded_model_json)
    # load weights into new model
    Reddit_model.load_weights("Models/model_Reddit.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Reddit_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Reddit_model.layers:
        print(layer, layer.trainable)
    Reddit_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Reddit_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Reddit_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fa0f8e180b8> False
<keras.layers.recurrent.LSTM object at 0x7fa0f8e181d0> True
<keras.layers.core.Dense object at 0x7fa0f8e18198> True
<keras.layers.core.Dense object at 0x7fa0f8b96e10> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fa0f8e180b8> False
<keras.layers.recurrent.LSTM object at 0x7fa0f8e181d0> False
<keras.layers.core.Dense object at 0x7fa0f8e18198> True
<keras.layers.core.Dense object at 0x7fa0f8b96e10> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8938492063492064, 0.9066534260178749, 0.9264413518886679, 0.9055666003976143, 0.9005964214711729]
Mean: 0.9066214012249073
Std Dev: 0.010893791054195214
Macro-F1-Scores: 
[0.8535009093712094, 0.8728890488365597, 0.8981803658963585, 0.8713165247248122, 0.860778973473274]
Mean: 0.8713331644604428
Std Dev: 0.01518271087108233
Without Fine Tuning: 
Accuracies: 
[0.873015873015873, 0.8887785501489573, 0.8906560636182903, 0.8916500994035785, 0.8747514910536779]
Mean: 0.8837704154480754
Std Dev: 0.008143514760871688
Macro-F1-Scores: 
[0.8088555987484592, 0.833280117311763, 0.8373269044188987, 0.8379804815342904, 0.810427135678392]
Mean: 0.8255740475383607
Std Dev: 0.013117684472013585
